In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# Load data

filename = 'haberman.data'
df = pd.read_csv(filename)
# Haberman survival dataset
# first column: pacient's age
# second column: year of operation
# third column: number nodes
# Fourth column: 1 (Surivved) - 2 (Died)
df.columns = ["Age","Year of operation","Nodes", "Result"]
df


,Age,Year of operation,Nodes,Result
0,30,62,3,1
1,30,65,0,1
2,31,59,2,1
3,31,65,4,1
4,33,58,10,1
...,...,...,...,...
300,75,62,1,1
301,76,67,0,1
302,77,65,3,1
303,78,65,1,2


In [10]:
#Treat data
# Because logistic regression is binary operator we have to 0 or 1 target values
# We will convert the 2's into 0's to represent that the pacient has died
df.loc[df['Result'] == 2, 'Result'] = 0
df

,Age,Year of operation,Nodes,Result
0,30,62,3,1
1,30,65,0,1
2,31,59,2,1
3,31,65,4,1
4,33,58,10,1
...,...,...,...,...
300,75,62,1,1
301,76,67,0,1
302,77,65,3,1
303,78,65,1,0


In [14]:
# Split between train and test sets
X = df[["Age","Year of operation", "Nodes"]]
X = np.array(X)
Y = df["Result"]
Y = np.array(Y)

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.33)
X_train

204 101
